In [1]:
from pydub import AudioSegment
import requests
from lxml import etree
from StringIO import StringIO

In [2]:
URL = 'https://www.youtube.com/watch?v=C-1mYXVP1Ok'

In [3]:
response = requests.get(URL)

In [4]:
parser = etree.HTMLParser()
tree = etree.parse(StringIO(response.text), parser)

In [5]:
start_times = tree.xpath(".//a[contains(@onclick, 'seekTo')]/text()")

In [9]:
track_names = [
    track.split(') ')[-1]
    for track in
    tree.xpath(".//a[contains(@onclick, 'seekTo')]/../text()")
    if track.strip()[0].isdigit() and ') ' in track
]

In [11]:
zip(start_times, track_names)

[('00:00', 'Rebel Rebel'),
 ('02:46', 'Life On Mars'),
 ('06:15', 'Starman'),
 ('09:32', 'Ziggy Stardust'),
 ('13:14', 'Lady Stardust'),
 ('16:45', 'Changes'),
 ('20:26', 'Oh! You Pretty Things'),
 ('23:58', "Rock N' Roll Suicide"),
 ('27:08', 'Suffragette City'),
 ('30:19', 'Five Years'),
 ('34:18', 'Queen Bitch'),
 ('38:00', 'When I Live My Dream'),
 ('40:55', 'Quicksand'),
 ('45:31', 'Team Zissou')]

In [12]:
from collections import namedtuple
Track = namedtuple('Track', 'name start_time end_time')

In [13]:
def parsetime(t):
    if t is None:
        return -1
    segments = map(int, t.split(':'))
    ms_mappings = [1000, 1000 * 60, 1000 * 60 * 60]
    return sum(
        value * conversion
        for value, conversion in
        zip(reversed(segments), ms_mappings)
    )

In [14]:
from itertools import izip_longest

tracks = [
    Track(name, parsetime(start_time), parsetime(end_time))
    for name, start_time, end_time in 
    izip_longest(track_names, start_times, start_times[1:])
]

In [15]:
tracks

[Track(name='Rebel Rebel', start_time=0, end_time=166000),
 Track(name='Life On Mars', start_time=166000, end_time=375000),
 Track(name='Starman', start_time=375000, end_time=572000),
 Track(name='Ziggy Stardust', start_time=572000, end_time=794000),
 Track(name='Lady Stardust', start_time=794000, end_time=1005000),
 Track(name='Changes', start_time=1005000, end_time=1226000),
 Track(name='Oh! You Pretty Things', start_time=1226000, end_time=1438000),
 Track(name="Rock N' Roll Suicide", start_time=1438000, end_time=1628000),
 Track(name='Suffragette City', start_time=1628000, end_time=1819000),
 Track(name='Five Years', start_time=1819000, end_time=2058000),
 Track(name='Queen Bitch', start_time=2058000, end_time=2280000),
 Track(name='When I Live My Dream', start_time=2280000, end_time=2455000),
 Track(name='Quicksand', start_time=2455000, end_time=2731000),
 Track(name='Team Zissou', start_time=2731000, end_time=-1)]

In [16]:
songs = AudioSegment.from_mp3('songs.mp3')

In [17]:
audio_tracks = [(t.name, songs[t.start_time:t.end_time]) for t in tracks]

In [18]:
import string
for i, (title, track) in enumerate(audio_tracks):
    track.export(
        '{:02d} - {}.mp3'.format(i, title),
        format='mp3',
        tags={
            'title': title,
            'album': 'The Life Aquatic Studio Sessions',
            'artist': 'Seu Jorge',
        }
    )

In [15]:
help(audio_tracks[0][1].export)

Help on method export in module pydub.audio_segment:

export(self, out_f=None, format='mp3', codec=None, bitrate=None, parameters=None, tags=None, id3v2_version='4') method of pydub.audio_segment.AudioSegment instance
    Export an AudioSegment to a file with given options
    
    out_f (string):
        Path to destination audio file
    
    format (string)
        Format for destination audio file.
        ('mp3', 'wav', 'raw', 'ogg' or other ffmpeg/avconv supported files)
    
    codec (string)
        Codec used to encoding for the destination.
    
    bitrate (string)
        Bitrate used when encoding destination file. (64, 92, 128, 256, 312k...)
        Each codec accepts different bitrate arguments so take a look at the
        ffmpeg documentation for details (bitrate usually shown as -b, -ba or
        -a:b).
    
    parameters (string)
        Aditional ffmpeg/avconv parameters
    
    tags (dict)
        Set metadata information to destination files
        usually us